# Profiling, Cython, and Numba 🚀
### Zbyszek & Jakob
### ASPP 2022, Bilbao, Spain

## Outline

* Introduction
* Profiling
* Speed up Python code using Cython
 * Basic principles
 * Interacting with NumPy arrays
* Using Numba to speed up Python code

 * ~Release the GIL and parallelize easily~ *(moved to parallel lecture)*
 * ~Wrap C/C++ code~ *(not relevant enough)*

## Introduction

* Sometimes, it seems like the execution speed of some script is *the* thing which keeps you from your next scientific breakthrough
* Both Cython and Numba are tools to make your code faster -> "optimization"
* So when should you optimize your code?

oral exercise: give examples in which scenarios you would benefit from optimization

## The three rules of optimization
(adapted from Sebastian Witowski, EuroPython 2016)

#### 1. Don't.
 * Optimization comes with costs.
 * Likely you don't need it.
 * Invest in better hardware.

oral exercise: give examples for costs associated with optimization

#### 2. Don't yet.
 * Is your code finished?
 * Did you write tests?
 * Are you sure it's worth the investment?

#### 3. Profile
* Don't guess which part of your code you should optimize!
* Measure. Measure. Measure.

## Runtime profilers

- profilers monitor the execution of your script and record, for example, how much time is spent in each function
- here we consider [py-spy](https://github.com/benfred/py-spy), a sampling-based profiler for Python
  - simply speaking `py-spy` examines your program after regular interval and records which part is currently executed
- you can apply it to your script with `py-spy record -o profile.svg -- python myprogram.py`
  - to make measurements accurate it needs to collect enough of data; you can control the "sampling rate" using the `-r` argument
- after measuring `py-spy` will produce a "flamegraph" like the following
![flamegraph](./figures/flamegraph.svg)

In [ ]:
!cat integrate_print.py

In [ ]:
%run integrate_print.py

In [ ]:
def f(x):
    return x ** 4 - 3 * x

In [ ]:
%timeit integrate_f(f, -10, +10, 1_000_000)

In [ ]:
%load_ext cython

In [ ]:
%%cython

def f(x):
    return x ** 4 - 3 * x

def integrate_f2(f, a, b, n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f(a) * dx2
    for i in range(1, n):
        s += f(a + i * dx) * dx
    s += f(b) * dx2
    return s


In [ ]:
f, integrate_f2

In [ ]:
sys.modules[f.__module__]

In [ ]:
!file /home/zbyszek/.cache/ipython/cython/_cython_magic_a9dc65ed82a290407cecd88aeb8605c0.cpython-310-x86_64-linux-gnu.so

In [ ]:
%timeit integrate_f2(f, -10, +10, 1_000_000)

In [ ]:
%%cython -a

def f(x):
    return x ** 4 - 3 * x

def integrate_f2(f, a, b, n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f(a) * dx2
    for i in range(1, n):
        s += f(a + i * dx) * dx
    s += f(b) * dx2
    return s


In [ ]:
%%cython -a

def f3(x: double) -> double:
    return x ** 4 - 3 * x

def integrate_f3(f, a: double, b: double, n: int) -> double:
    dx: double = (b - a) / n
    dx2: double = dx / 2
    s: double = f(a) * dx2
    i: int = 0

    for i in range(1, n):
        s += f(a + i * dx) * dx
    s += f(b) * dx2
    return s


In [ ]:
%timeit integrate_f3(f3, -10, +10, 1_000_000)

In [ ]:
%%cython -a

def f3a(double x):
    return x ** 4 - 3 * x

def integrate_f3a(f, double a, double b, int n) -> double:
    cdef:
        double dx = (b - a) / n
        double dx2 = dx / 2
        double s
        int i

    s = f(a) * dx2
    for i in range(1, n):
        s += f(a + i * dx) * dx
    s += f(b) * dx2
    return s


In [ ]:
%timeit integrate_f3a(f3a, -10, +10, 1_000_000)

In [ ]:
%%cython -a

cdef double f4(double x):
    return x ** 4 - 3 * x

def integrate_f4(double a, double b, int n) -> double:
    cdef:
        double dx = (b - a) / n
        double dx2 = dx / 2
        double s
        int i

    s = f4(a) * dx2
    for i in range(1, n):
        s += f4(a + i * dx) * dx
    s += f4(b) * dx2
    return s


In [ ]:
%timeit integrate_f4(-10, +10, 1_000_000)

In [ ]:
%timeit integrate_f3(f3, -10, +10, 1_000_000)

In [ ]:
%%cython?

In [ ]:
def integrate_f0(f, a, b, n):
    s = 0.0
    for i in range(n):
        dx = (b - a) / n
        x = a + (i + 0.5) * dx
        s += f(x) * dx
    return s

In [ ]:
def integrate_f1(f, a, b, n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        x = a + (i + 0.5) * dx
        s += f(x) * dx
    return s

In [ ]:
def integrate_f2(f, a, b, n):
    dx = (b - a) / n
    x = a + 0.5 * dx
    s = 0.0
    for i in range(n):
        s += f(x) * dx
        x += dx
    return s

In [ ]:
def integrate_f3(f, a, b, n):
    dx = (b - a) / n
    x = a + 0.5 * dx
    s = 0.0
    for i in range(n):
        s += f(x)
        x += dx
    return s * dx

demo: explain how to read the flamegraph using a simple example

In [ ]:
## Example: numerical integration

![RiemannSum](figures/MidRiemann2.svg)

Riemann sum: $\int_a^b dx f(x) \approx \sum_{i = 0}^{n - 1} f(a + (i + 0.5) \Delta x) \Delta x$ with $\Delta x = (b - a)/n$

here $a=0, b=2, n=4$

### Example implementation
(see [./profiling/numerical_integration.py](./profiling/numerical_integration.py))

Where do you think the bottlenecks are? *(don't do this at home!)*

## Demo

Jakob will demonstrate a typical profiling/optimization workflow based on this script.

- time
- py-spy
- notebook (timeit/lprun)
- time (of improved version)

## Exercise

It's time to put theory into practice. We have prepared an example script (see [./profiling/count_words.py](./profiling/count_words.py)) which counts the number of occurences of words in a text.

1. Familarize yourself with the script.
2. Guess which parts are slow and should be optimized. *(don't do this at home.)*
3. Use the workflow (time -> py-spy- > timeit/lprun -> time) we have just demonstrated to reduce the script's execution time. **Make sure not to break the tests.**

Afterwards we will discuss the exercise jointly.

## Exercise discussion

What did we learn?
- ...

## Profiling conclusion

- before optimizing, first finish your code & write tests
- then *measure* to find slow functions
- optimize only the slowest functions & know when to stop!
- [py-spy](https://github.com/benfred/py-spy) is just one of many profilers; alternatives:
  - [cProfile](https://docs.python.org/3/library/profile.html) + [snakeviz](https://github.com/jiffyclub/snakeviz)
  - [pyinstrument](https://github.com/joerick/pyinstrument)
- here we focus on profiling *runtime*, but maybe you are limited by *memory*
  - [memray](https://github.com/bloomberg/memray)

- 80/20 rule
- datastructures and algorithms
- libraries
- memoization/caching

In [ ]:
Profiling

- %timeit
- cProfile
- pyprof2calltree + kcachegrind
- gprof2dot ?
- kernprof ?
- py-spy, pyinstrument ?



